In [1]:
import girder_client
%load_ext autoreload
%autoreload 2

import pandas as pd
import adrcSchemaHelpers as hlprs

gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")

Unknown Stain Tags: []
Unmatched File Names: []


In [3]:
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '638e2ae7cd1d7fad17b9dd1f',
 '_modelType': 'user',
 'admin': True,
 'created': '2022-12-05T17:31:19.547000+00:00',
 'email': 'admin@nowhere.nil',
 'emailVerified': True,
 'firstName': 'Admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'Admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 0,
 'status': 'enabled'}

In [ ]:
folders = [
    folder
    for folder in gc.listFolder('638e2da11f75016b81fda12f', parentFolderType='collection')
]


  #  _id: internal mongo id
  #  _modelType: object type (folder, item, or collection -- also annotations but not so relevant)
  #  baseParentId: collection the item is in
  #  baseParentType: parent object type (folder, item, or collection -- also annotations but not so relevant)
  #  created: date created
  #  creatorId: id of individual who created the item
  #  description: description of the item
  #  meta: dictionary of meta data pertaining to item
  #  name: item name (may be a year, patient id, or other folder/file names)
  #  parentCollection: *unclear, but told to ignore*
  #  parentId: id of parent collection
  #  public: whether item is public or not
  #  size: size in bytes (or other unit) -- folders show as 0
  #  updated: datetime of last update


In [ ]:
baseURL = "resource/*/items?type=folder&limit=1000"

itemSet = [
    item["meta"]["npSchema"]
    for folder in folders
    for subFolder in gc.listFolder(folder["_id"])
    for item in gc.getResource(baseURL.replace("*", subFolder['_id']))
    if item["meta"].get("npSchema") is not None
]

# So just to confuse things, I don't necessarily have to have a single folder below each case Folder
# To make things easy, I can just get all the items though, as I don't really care about what sub sub folder things are in
# See here
# http://candygram.neurology.emory.edu:8080/api/v1#!/resource/resource_getResourceItems_id_items


In [ ]:
allVals = {}

for item in itemSet:
    for (key, val) in item.items():
        if allVals.get(key) is None:
            allVals[key] = [val]
        elif val not in allVals[key]:
            allVals[key].append(val)

In [ ]:
lens = [len(val) for val in allVals.values()]
lens = [max(lens) - length for length in lens]

In [ ]:
# for pandas to make df from dict, columns must be of equal length, so adding "val" number of Nones
for key, val in zip(allVals.keys(), lens):
    if val:
        allVals[key].extend([None] * val)

In [ ]:
df = pd.DataFrame.from_dict(allVals)
df["stainMap"] = None

In [ ]:
stainList = ['ptdp', 'he', 'abeta', 'ubiq', "tau", "biels", "syn", "p62", "lfb"]
filt = (~df["stainID"].isin(stainList)) & (df["stainID"].notna())

df.loc[filt, "stainMap"] = df.loc[filt, "stainID"].map((lambda x: any([x.lower() in val for val in stainList])))

In [ ]:
filt = df["stainMap"].notna()
mappable = df.loc[filt, "stainID"].unique().tolist()

mappable = {key: val for key in mappable for val in stainList if val in key.lower()}
df["stainID"].replace(mappable, inplace=True)

In [ ]:
df.drop(columns=["stainMap"], inplace=True)

for col in df.columns:
    counted = df[col].value_counts()
    counted.name = "Count"
    counted.index.name = col
    counted.to_csv(f"./{col}_vals_counted.csv")

In [ ]:
df.drop_duplicates(subset=["stainID"], inplace=True)

df.to_csv("./all_vals.csv", index=False)

In [4]:
sampPatient = ['638e33521f75016b81fdb8d2']  ## 2018 / E18-07

In [4]:
for f in gc.listFolder('638e2da11f75016b81fda12f',parentFolderType='collection'):
    for sf in gc.listFolder(f['_id']):
        r,_ = hlprs.scanMetadata(gc, sf, updateGirder=True)
        if r["girderUpdates"] or r["slidesToUpdate"]:
            print(r)


Needs Brain Map: {'96-33'}
Needs Brain Map: {'97-53'}
Needs Brain Map: {'E97-60'}
Needs Brain Map: {'E99-04'}
Needs Brain Map: {'E99-07'}
Needs Brain Map: {'99-26'}
Needs Brain Map: {'99-30'}
Needs Brain Map: {'00-06'}
Needs Brain Map: {'E00-10'}
Needs Brain Map: {'00-11'}
Needs Brain Map: {'00-12'}
Needs Brain Map: {'E00-14'}
Needs Brain Map: {'00-18'}
Needs Brain Map: {'00-19', 'E00-19'}
Needs Brain Map: {'E00-21', '00-21'}
Needs Brain Map: {'00-28'}
Needs Brain Map: {'00-32'}
Needs Brain Map: {'E00-40'}
Needs Brain Map: {'00-41'}
Needs Brain Map: {'E01-06'}
Needs Brain Map: {'01-134', 'E01-134'}
Needs Brain Map: {'E01-152', '51-162'}
Needs Brain Map: {'01-33', 'E01-33', 'E01-62', '33-1'}
Needs Brain Map: {'01-60', '51-60'}
Needs Brain Map: {'01-70'}
Needs Brain Map: {'01-60'}
Needs Brain Map: {'02-207'}
Needs Brain Map: {'02-208'}
Needs Brain Map: {'02-35'}
Needs Brain Map: {'03-385'}
Needs Brain Map: {'03-163'}
Needs Brain Map: {'03-299'}
Needs Brain Map: {'53-300'}
Needs Brain Map

Needs Block Map: {'E11-41': {'1': 98}}
{'Processed': 98, 'girderUpdates': 86, 'slidesToUpdate': 86}
Needs Block Map: {'E11-59': {'1': 90}}
{'Processed': 92, 'girderUpdates': 87, 'slidesToUpdate': 87}
Needs Block Map: {'E11-60': {'2': 69, '11': 1, '13': 2, '15': 2, '17': 1, '19': 2, '5': 1, '7': 2, '9': 2}}
{'Processed': 97, 'girderUpdates': 63, 'slidesToUpdate': 63}
Needs Block Map: {'E11-70': {'A1': 7, 'A2': 9, 'A3': 7, 'A4': 8, 'A10': 5, 'A11': 4, 'A12': 3, 'A13': 3, 'A14': 3, 'A15': 5, 'A16': 5, 'A17': 5, 'A18': 4, 'A21': 4, 'A22': 4, 'A23': 1, 'A24': 1, 'A5': 5, 'A6': 5, 'A7': 3, 'A8': 3, 'A9': 5}}
{'Processed': 99, 'girderUpdates': 5, 'slidesToUpdate': 5}
Needs Block Map: {'E11-71': {'A1': 7, 'A3': 7, 'A10': 4, 'A11': 4, 'A12': 3, 'A13': 3, 'A14': 3, 'A15': 4, 'A16': 5, 'A17': 5, 'A18': 4, 'A19': 6, 'A20': 6, 'A21': 4, 'A22': 4, 'A23': 1, 'A24': 1, 'A2': 6, 'A4': 6, 'A5': 4, 'A6': 4, 'A7': 3, 'A8': 3, 'A9': 4}}
{'Processed': 105, 'girderUpdates': 65, 'slidesToUpdate': 65}
Needs Bl

{'Processed': 104, 'girderUpdates': 2, 'slidesToUpdate': 2}
Needs Brain Map: {'17-53'}
{'Processed': 165, 'girderUpdates': 1, 'slidesToUpdate': 1}
{'Processed': 89, 'girderUpdates': 5, 'slidesToUpdate': 5}
Needs Brain Map: {'E18-7'}
{'Processed': 77, 'girderUpdates': 1, 'slidesToUpdate': 1}
{'Processed': 64, 'girderUpdates': 1, 'slidesToUpdate': 1}
Needs Brain Map: {'E18-127'}
{'Processed': 76, 'girderUpdates': 5, 'slidesToUpdate': 5}
Needs Brain Map: {'E18-145'}
{'Processed': 36, 'girderUpdates': 8, 'slidesToUpdate': 8}
{'Processed': 81, 'girderUpdates': 1, 'slidesToUpdate': 1}
{'Processed': 76, 'girderUpdates': 4, 'slidesToUpdate': 4}
{'Processed': 143, 'girderUpdates': 2, 'slidesToUpdate': 2}
Needs Brain Map: {'E19-100'}
{'Processed': 95, 'girderUpdates': 3, 'slidesToUpdate': 3}
Needs Brain Map: {'E19-101'}
{'Processed': 88, 'girderUpdates': 3, 'slidesToUpdate': 3}
Needs Brain Map: {'E19-116'}
{'Processed': 141, 'girderUpdates': 6, 'slidesToUpdate': 6}
{'Processed': 92, 'girderUpdat